In [40]:
import pickle
import pandas as pd

In [41]:
with open("urls_to_download_fr.pkl", 'rb') as file:
    df = pickle.load(file)

In [42]:
df["is_fake"].value_counts()

is_fake
0    1630
1      22
Name: count, dtype: int64

In [43]:
df[df["is_fake"]==1].links

309     https://fr.sputniknews.com/20220101/une-folle-...
311     https://fr.sputniknews.com/20220101/le-canada-...
316     https://fr.sputniknews.com/20220101/apres-des-...
485     https://fr.sputniknews.com/20220101/ukraine-bi...
520     https://fr.sputniknews.com/20220101/covid-19om...
540     https://fr.sputniknews.com/20220101/inde-12-mo...
542     https://fr.sputniknews.com/20220101/la-marine-...
713     https://fr.sputniknews.com/20220101/le-drapeau...
718     https://fr.sputniknews.com/20220101/une-ex-con...
823     https://fr.sputniknews.com/20220101/boris-john...
827     https://fr.sputniknews.com/20220101/lafrique-d...
910     https://fr.sputniknews.com/20220101/la-hausse-...
941     https://fr.sputniknews.com/20220101/lukraine-f...
1121    https://fr.sputniknews.com/20220101/autopromot...
1154    https://fr.sputniknews.com/20220101/une-rare-p...
1157    https://fr.sputniknews.com/20220101/gabon-les-...
1160    https://fr.sputniknews.com/20220101/le-niger-d...
1177    https:

In [46]:
df.iloc[309].links

'https://fr.sputniknews.com/20220101/une-folle-experience-positive-au-covid-en-plein-vol-une-americaine-passe-5-heures-aux-toilettes-1054138351.html'